# Explore here

In [146]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from flask import Flask, request, render_template
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pickle

In [147]:
file_path = "Netflix TV Shows and Movies.csv"

In [148]:
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "sufyan145/netflix-movies-and-shows-imdb-scores",
  file_path,
)


C:\Users\juanm\AppData\Local\Temp\ipykernel_2872\663258750.py:1: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [149]:
df

,index,id,title,type,description,release_year,age_certification,runtime,imdb_id,imdb_score,imdb_votes
0,0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,tt0075314,8.3,795222.0
1,1,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,tt0071853,8.2,530877.0
2,2,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,tt0079470,8.0,392419.0
3,3,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,tt0070047,8.1,391942.0
4,4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,tt0063929,8.8,72895.0
...,...,...,...,...,...,...,...,...,...,...,...
5278,5278,tm1040816,Momshies! Your Soul is Mine,MOVIE,Three women with totally different lives accid...,2021,NaN,108,tt14412240,5.8,26.0
5279,5279,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,tt13857480,6.9,39.0
5280,5280,tm1045018,Clash,MOVIE,A man from Nigeria returns to his family in Ca...,2021,NaN,88,tt14620732,6.5,32.0
5281,5281,tm1098060,Shadow Parties,MOVIE,A family faces destruction in a long-running c...,2021,NaN,116,tt10168094,6.2,9.0


In [150]:
df.shape

(5283, 11)

Limpiamos valores nulos.

In [151]:
df.dropna(inplace=True)

In [152]:
df.shape

(2987, 11)

PAsamos valores de texto a numerico

In [153]:
le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])

Variables para predecir

In [154]:
X = pd.get_dummies(df[['imdb_votes', 'age_certification']], drop_first=True)
y = df['type_encoded']

Divdimos en train y test

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [156]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [157]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       277
           1       1.00      1.00      1.00       321

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



Guardar modelos

In [158]:
pickle.dump(clf, open('modelo_rf.pkl', 'wb'))

pickle.dump(le, open('encoder.pkl', 'wb'))

In [159]:
app = Flask(__name__)

In [ ]:
model = pickle.load(open('modelo_rf.pkl', 'rb'))
encoder = pickle.load(open('encoder.pkl', 'rb'))

@app.route('/', methods=['GET', 'POST'])
def predict():
    resultado = None
    if request.method == 'POST':
        imdb_votes = int(request.form['imdb_votes'])
        age_certification = request.form['age_certification']
       
        X_input = pd.DataFrame({'imdb_votes': [imdb_votes], 'age_certification': [age_certification]})
        X_input = pd.get_dummies(X_input, columns=['age_certification'], drop_first=True)

        for col in model.feature_names_in_:
            if col not in X_input.columns:
                X_input[col] = 0
        X_input = X_input[model.feature_names_in_]
        pred = model.predict(X_input)
        resultado = encoder.inverse_transform(pred)[0]
    return render_template('index.html', resultado=resultado)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2025-07-21 00:11:37,887] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\juanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\juanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\juanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\juanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\juanm\AppData\Local\Te